In [ ]:
!pip install deepagents tavily-python
!pip install tavily
!pip install deepagents

### Define model and set api keys

In [18]:
from langchain.tools import tool
from langchain.chat_models import init_chat_model

import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import getpass

load_dotenv()

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")

model = init_chat_model("google_genai:gemini-2.5-flash-lite",
                        temperature = 0.7,
                        timeout=30,
                        max_tokens=1000)

## Create Search Tool

In [19]:
import os
from typing import Literal
from tavily import TavilyClient
from deepagents import create_deep_agent

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

## Create an Agent

In [22]:
# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher. Your job is to conduct thorough research and then write a polished report.
You have access to an internet search tool as your primary means of gathering information.
## `internet_search`
Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.
"""

agent = create_deep_agent(
    model=model,
    tools=[internet_search],
    system_prompt=research_instructions
)

### Run Agent

In [34]:
result = agent.invoke({"messages": [{"role": "user", "content": "Explain transformer model."}]})

from pprint import pprint
# Print the agent's response
pprint(result["messages"][-1].content)

('The Transformer model is a type of neural network architecture that has '
 'revolutionized natural language processing (NLP) tasks. It was introduced in '
 'the paper "Attention Is All You Need" by Vaswani et al. in 2017.\n'
 '\n'
 "Here's a breakdown of its key components and concepts:\n"
 '\n'
 '**1. Architecture:**\n'
 '   - The Transformer architecture consists of two main parts: an **encoder** '
 'and a **decoder**.\n'
 '   - Both the encoder and decoder are typically composed of multiple '
 'identical layers stacked on top of each other.\n'
 '\n'
 '**2. Core Idea: Self-Attention:**\n'
 '   - The most significant innovation of the Transformer is the '
 '**self-attention mechanism**.\n'
 '   - Unlike previous models (like RNNs and LSTMs) that processed sequences '
 'word-by-word, self-attention allows the model to weigh the importance of '
 'different words in the input sequence when processing a particular word.\n'
 '   - This means the model can capture long-range dependencies 